In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
#import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

In [2]:
datadir = 'input/'
gatrain = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')
gatest = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv(os.path.join(datadir,'events.csv'),
                     parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv(os.path.join(datadir,'app_events.csv'), 
                        usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))

In [3]:
gatrain['trainrow'] = np.arange(gatrain.shape[0])
gatest['testrow'] = np.arange(gatest.shape[0])

In [4]:
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.brand)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_brand.shape, Xte_brand.shape))

Brand features: train shape (74645, 131), test shape (112071, 131)


In [5]:
m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.model)))
print('Model features: train shape {}, test shape {}'.format(Xtr_model.shape, Xte_model.shape))

Model features: train shape (74645, 1667), test shape (112071, 1667)


In [22]:
Xtrain = hstack((Xtr_brand, Xtr_model), format='csr')
Xtest =  hstack((Xte_brand, Xte_model), format='csr')
print('All features: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

All features: train shape (74645, 1798), test shape (112071, 1798)


In [24]:
X = Xtrain.toarray()

In [25]:
Xtrain = csr_matrix(X)

In [26]:
Xtrain

<74645x1798 sparse matrix of type '<class 'numpy.float64'>'
	with 149290 stored elements in Compressed Sparse Row format>

In [13]:
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

In [18]:
def score(clf, random_state = 22):
    kf = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=random_state)
    pred = np.zeros((y.shape[0],nclasses))
    for itrain, itest in kf:
        Xtr, Xte = Xtrain[itrain, :], Xtrain[itest, :]
        ytr, yte = y[itrain], y[itest]
        clf.fit(Xtr, ytr)
        pred[itest,:] = clf.predict_proba(Xte)
        # Downsize to one fold only for kernels
        print("{:.5f}".format(log_loss(yte, pred[itest,:])), end=' ')
        return log_loss(yte, pred[itest, :])
    #print('')
    print("{:.5f}".format(log_loss(y, pred)), end=' ')
    return pred

In [27]:
preds = score(LogisticRegression(C=0.11666667, multi_class='multinomial',solver='lbfgs'))

2.38815 

In [15]:
pred = pd.DataFrame(preds, index = gatrain.index, columns=targetencoder.classes_)
pred.head()

,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
-8076087639492063270,0.037475,0.067798,0.041063,0.065220,0.051750,0.039853,0.069456,0.166081,0.107127,0.107441,0.147807,0.098929
-2897161552818060146,0.032803,0.071605,0.041575,0.063344,0.052352,0.038962,0.069328,0.166768,0.106016,0.108658,0.152392,0.096199
-8260683887967679142,0.033982,0.072932,0.037267,0.061902,0.052708,0.039218,0.062273,0.165117,0.109896,0.107058,0.152369,0.105279
-4938849341048082022,0.056571,0.058165,0.041428,0.069147,0.082090,0.067195,0.085281,0.112692,0.071051,0.101413,0.130889,0.124078
245133531816851882,0.066222,0.076016,0.053673,0.061993,0.076234,0.046234,0.123724,0.135783,0.075291,0.097531,0.097494,0.089806


In [18]:
log_loss(y,np.array(pred))

2.3907051904597698

In [19]:
pred.to_csv('logreg_brand_model_train.csv',index=True)